# Puntos de donación de sangre de la Comunidad de Madrid

In [1]:
import json

import pandas as pd

from tqdm.auto import tqdm
from dotenv import load_dotenv

from utils.db import format_column, from_db, gdf_from_df, run_query, to_db
from utils.geocode import google_geocode_address, get_full_address
from utils.crawl import extraer_puntos_fijos, extraer_puntos_moviles, get_gmaps_url

In [2]:
# Read env variables from .env for local development
# In Cloud Functions, env variables must be set in runtime environment directly
load_dotenv()

with open('../sql/table_to_geojson.sql') as fp:
    TABLE_TO_GEOJSON_QUERY = fp.read()

## Puntos fijos

Los hospitales son fáciles de scrapear pero hay otros tres puntos que aparecen desordenados en [la web](https://www.comunidad.madrid/servicios/salud/donacion-sangre#puntos-donacion).

Los metemos a mano

In [3]:
with open("utils/puntos_fijos_no_hospitales.json", "rt", encoding="utf-8") as fp:
    puntos_fijos_no_hospitales = json.load(fp)

# Puntos Fijos

Juntamos los pocos que no salen en la web

In [4]:
df_fijos = extraer_puntos_fijos()
df_fijos_keys = pd.DataFrame({'original': df_fijos.columns, 'db': df_fijos.columns.map(format_column)})
df_fijos.columns = df_fijos.columns.map(format_column)
df_fijos = pd.concat([pd.DataFrame(puntos_fijos_no_hospitales), df_fijos], ignore_index=True)
df_fijos['gmaps_url'] = df_fijos.apply(get_gmaps_url, axis=1)

gdf_fijos = gdf_from_df(df_fijos)

Tenemos que eliminar y recrear la vista de `geojson` dependiente de la tabla de puntos fijos

In [6]:
run_query('DROP VIEW IF EXISTS puntos_fijos_geojson')

to_db(gdf_fijos, 'puntos_fijos')
to_db(df_fijos_keys, 'puntos_fijos_keys')


run_query(TABLE_TO_GEOJSON_QUERY.format(t='puntos_fijos'))

## Puntos móviles

Muy similar a los puntos fijos, sólo que no tienen web individual y pillamos todos los atributos de una lista

In [7]:
df_moviles = extraer_puntos_moviles()
df_moviles.columns = df_moviles.columns.map(format_column)

for text in ['Equipo móvil en ', 'E Móvil en ', 'Equipo Móvil detrás ']:
    df_moviles['direccion'] = df_moviles['direccion'].map(lambda d: d.replace(text, ''))

Geocode and upload. We geocode both the `lugar` and `direccion` and keep their scores
based on the `location_type` (ROOFTOP, etc).

Before geocoding, try to use the cache

In [8]:
try:
    geocoding_cache = from_db('geocoding_cache')
except Exception:
    geocoding_cache = pd.DataFrame(columns=['address', 'longitude', 'latitude', 'location_type', 'score'])

In [9]:
new_geocoding_list = []
for address_field in ['lugar', 'direccion']:
    full_addresses = df_moviles.apply(get_full_address, axis=1, address_field=address_field)
    for full_address in tqdm(full_addresses.drop_duplicates(), desc=address_field):
        # Try to find address in cache
        gc_row = geocoding_cache.loc[geocoding_cache.address == full_address]
        if not gc_row.empty:
            continue

        # Address not found. Must geocode
        print(f'"{full_address}" not found in cache. Geocoding...')
        lng, lat, location_type, score = google_geocode_address(full_address)
        new_geocoding_list.append({
            'address': full_address,
            'longitude': lng,
            'latitude': lat,
            'location_type': location_type,
            'score': score,
        })

new_geocoding_df = pd.DataFrame(new_geocoding_list)
geocoding_cache = pd.concat([geocoding_cache, new_geocoding_df])

geocoding_cache = geocoding_cache.drop_duplicates()

to_db(geocoding_cache, 'geocoding_cache')

lugar:   0%|          | 0/84 [00:00<?, ?it/s]

"PROCTER, Alcobendas, Community of Madrid, Spain" not found in cache. Geocoding...
"INDRA- ALCOBENDAS Arroyo de la Vega, Alcobendas, Community of Madrid, Spain" not found in cache. Geocoding...
"Pª Santa María de la Esperanza, Alcobendas, Community of Madrid, Spain" not found in cache. Geocoding...
"Avenida de la Libertad (Metro Sur), Alcorcón, Community of Madrid, Spain" not found in cache. Geocoding...
"Algete (URB. STO DOMINGO), Algete, Community of Madrid, Spain" not found in cache. Geocoding...
"C/ Stuart, 91 (Ayuntamiento), Aranjuez, Community of Madrid, Spain" not found in cache. Geocoding...
"C/ Juan Carlos I, 40 (varios puntos), Boadilla del Monte, Community of Madrid, Spain" not found in cache. Geocoding...
"BUITRAGO DEL LOZOYA, Buitrago del Lozoya, Community of Madrid, Spain" not found in cache. Geocoding...
"CERCEDA, El Boalo, Community of Madrid, Spain" not found in cache. Geocoding...
"C. CO LORANCA, Fuenlabrada, Community of Madrid, Spain" not found in cache. Geocoding..

direccion:   0%|          | 0/88 [00:00<?, ?it/s]

"Av. de Bruselas, 24, Alcobendas, Community of Madrid, Spain" not found in cache. Geocoding...
"Av. de Bruselas, 33-35, Alcobendas, Community of Madrid, Spain" not found in cache. Geocoding...
"C/ Ramón y Cajal, 3 junto Pª Santa María de la Esperanza, Alcobendas, Community of Madrid, Spain" not found in cache. Geocoding...
"Avenida de la Libertad frente Metro Puerta del Sur, Alcorcón, Community of Madrid, Spain" not found in cache. Geocoding...
"Plaza de la Constitución, Algete, Community of Madrid, Spain" not found in cache. Geocoding...
"Pza de la Constitución (Ayuntamiento), Aranjuez, Community of Madrid, Spain" not found in cache. Geocoding...
"C/ Juan Carlos I, 40, Boadilla del Monte, Community of Madrid, Spain" not found in cache. Geocoding...
"Plaza del Ayuntamiento, El Boalo, Community of Madrid, Spain" not found in cache. Geocoding...
"Avenida Pablo Iglesias esquina C/ Madres de Mayo, Fuenlabrada, Community of Madrid, Spain" not found in cache. Geocoding...
"avda de las ciudad

In [10]:
# Geocoding candidates based on "lugar"
df_ubicacion = df_moviles[['lugar', 'localidad']].copy()
df_ubicacion['address'] = df_ubicacion.apply(get_full_address, axis=1, address_field='lugar')
df_ubicacion = pd.merge(df_ubicacion, geocoding_cache, on='address')

# Geocoding candidates based on "direccion"
df_direccion = df_moviles[['direccion', 'localidad']].copy()
df_direccion['address'] = df_direccion.apply(get_full_address, axis=1, address_field='direccion')
df_direccion = pd.merge(df_direccion, geocoding_cache, on='address')

assert len(df_moviles) == len(df_ubicacion) == len(df_direccion)

Get the coordinates from the best candidate address (`direccion` or `lugar`)

In [11]:
df_moviles['latitude'] = None
df_moviles['longitude'] = None
for idx, row in df_moviles.iterrows():
    score_ubicacion = df_ubicacion.loc[idx, 'score']
    score_direccion = df_direccion.loc[idx, 'score']
    if score_direccion > score_ubicacion:
        lng = df_direccion.loc[idx, 'longitude']
        lat = df_direccion.loc[idx, 'latitude']
    else:
        lng = df_ubicacion.loc[idx, 'longitude']
        lat = df_ubicacion.loc[idx, 'latitude']
        
    df_moviles.loc[idx, 'longitude'] = lng
    df_moviles.loc[idx, 'latitude'] = lat

In [12]:
db_fields = ['nombre', 'localidad', 'direccion', 'fecha', 'horario']
nice_fields = ['Nombre', 'Localidad', 'Dirección', 'Fecha', 'Horario']
df_moviles_keys = pd.DataFrame({'original': nice_fields, 'db': db_fields})

df_moviles['url'] = df_moviles.apply(get_gmaps_url, axis=1)
gdf_moviles = gdf_from_df(df_moviles)
gdf_moviles = gdf_moviles.drop(columns=['lugar'])
gdf_moviles['nombre'] = 'Equipo móvil en ' + df_moviles['lugar']
gdf_moviles = gdf_moviles[db_fields + ['url', 'geometry']]

In [13]:
run_query('DROP VIEW IF EXISTS puntos_moviles_geojson')

to_db(gdf_moviles, 'puntos_moviles')
to_db(df_moviles_keys, 'puntos_moviles_keys')

run_query(TABLE_TO_GEOJSON_QUERY.format(t='puntos_moviles'))